In [7]:
import os
import numpy as np
import faiss
import ollama
from tqdm import tqdm
import pickle

In [8]:


# 步骤2: 配置常量
EMBEDDING_MODEL = "nomic-embed-text"
VECTOR_STORE_DIR = "./vector_stores"
INDEX_NAME = "my_documents"
MODEL_NAME = "llama3.2:latest" 

# 步骤3: 加载向量存储
def load_vector_store(index_name):
    index_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}.faiss")
    metadata_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}_metadata.pkl")
    
    try:
        index = faiss.read_index(index_path)
        with open(metadata_path, "rb") as f:
            data = pickle.load(f)
            chunks = data['chunks']
            metadata = data['metadata']
        print(f"✅ 成功加载向量存储 | 索引大小: {index.ntotal}")
        return index, chunks, metadata
    except Exception as e:
        print(f"❌ 加载失败: {str(e)}")
        return None, None, None

# 加载向量存储
index, chunks, metadata = load_vector_store(INDEX_NAME)

# 步骤4: 定义查询函数
def query_documents(question, index, chunks, metadata, model_name=MODEL_NAME, k=3):
    """查询文档并获取回答"""
    try:
        # 获取问题嵌入
        response = ollama.embeddings(model=EMBEDDING_MODEL, prompt=question)
        query_embedding = np.array([response['embedding']], dtype=np.float32)
        
        # 搜索相似内容
        distances, indices = index.search(query_embedding, k)
        
        # 获取相关文本块
        context_chunks = [chunks[i] for i in indices[0]]
        context_metadata = [metadata[i] for i in indices[0]]
        
        # 构造上下文
        context = "\n\n".join([
            f"来源: {meta['filename']} 第{meta['page']}页\n内容: {chunk}"
            for chunk, meta in zip(context_chunks, context_metadata)
        ])
        
        # 构造提示
        prompt = f"""
        基于以下上下文信息回答问题：
        {context}
        
        问题: {question}
        回答:
        """
        
        # 获取回答
        response = ollama.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        return response['message']['content'], context_metadata
        
    except Exception as e:
        return f"查询失败: {str(e)}", []

# 步骤5: 交互式文档查询
print("🚀 文档助手已启动! 输入 'exit' 退出")

✅ 成功加载向量存储 | 索引大小: 202
🚀 文档助手已启动! 输入 'exit' 退出


In [ ]:
question = "what is time llm"
answer, sources = query_documents(question, index, chunks, metadata)

print(f"🤖 助手回答:\n{answer}\n")
print("📚 来源:")
for i, source in enumerate(sources):
    print(f"{i+1}. {source['filename']} - 第{source['page']}页")

🤖 助手回答:
TIME-LLM是通过重编程大型语言模型来实现时间系列 forecasting 的方法。它通过将时间系列数据转换为自然语言的prototype，并利用 Prompt-as-Prefix 批量激活 LLMs，以实现高性能的时间 series forecasting。

📚 来源:
1. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第6页
2. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第9页
3. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第5页


In [14]:
question = "time llm vs tab pfn, what is the difference"
answer, sources = query_documents(question, index, chunks, metadata)

print(f"🤖 助手回答:\n{answer}\n")
print("📚 来源:")
for i, source in enumerate(sources):
    print(f"{i+1}. {source['filename']} - 第{source['page']}页")

🤖 助手回答:
根据提供的文本信息，TIME-LLM（Time Series Forecasting using Large Language Models）和TabPFN（Transformer That Solves Small Tabular Classification Problems in a Second）是两个不同的机器学习模型。

TIME-LLM 是用于时间序列预测的Large Language Model（LLM），它通过使用Llama-7B作为底层模型，结合 transformer技术来实现时间序列预测。它在各种基线和竞争性模型上表现出卓越的效果，尤其是在少shot和零-shot场景中。

TabPFN 则是用于表格分类任务的Transformer模型，它通过对特定类型的表格数据进行训练，展示出在快速运行和调优阶段都能获得优异的性能。它在数字数据中表现最强大，并且在长时间运行期间也取得了良好的结果。

比较TIME-LLM和TabPFN的主要区别在于它们是用于不同类型的问题的模型：

* TIME-LLM 是用于时间序列预测的模型。
* TabPFN 是用于表格分类任务的模型。

尽管两者都使用Transformer技术，但它们有不同的设计和训练目标。TIME-LLM 的重点是利用大型语言模型来解决时间序列预测的问题，而TabPFN 的重点则是在快速运行和调优阶段获得高性能。

📚 来源:
1. TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf - 第22页
2. TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf - 第33页
3. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第6页
